In [ ]:
train_source = open('/content/drive/MyDrive/Copy of train.txt.src.tokenized.fixed.cleaned.final.truncated').read()

In [ ]:
train_source = train_source.split('\n')

In [ ]:
train_source.pop()

''

In [ ]:
train_target = open('/content/drive/MyDrive/Copy of train.txt.tgt.tokenized.fixed.cleaned.final.truncated').read()

In [ ]:
train_target = train_target.split("\n– ")

In [ ]:
print(len(train_source))
len(train_target)

44972


44972

In [ ]:
!pip install rouge

In [ ]:
!pip install sentence_transformers

In [ ]:
import nltk
from nltk import sent_tokenize,word_tokenize
nltk.download('punkt')
import numpy as np
from sentence_transformers import SentenceTransformer
import math
from rouge import Rouge 
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:20<00:00, 20.3MB/s]


In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
def convert_to_embeddings(hypothesis):
  text = hypothesis
  hypothesis_embeddings = {}
  hypothesis = sent_tokenize(hypothesis)
  sum = 0
  for sent in hypothesis:
    var = sbert_model.encode([sent])[0]
    sum+=var
    hypothesis_embeddings[sent] = var

  centroid = sum/len(hypothesis)
  #print("done")
  #print(hypothesis_embeddings)
  content_relevance_score = content_relevance(centroid,hypothesis_embeddings)
  sentence_novelty_score = sentence_novelty(content_relevance_score,hypothesis_embeddings)
  sentence_position_score = sentence_position(hypothesis)
  return total_score(content_relevance_score,sentence_novelty_score,sentence_position_score,text)

In [ ]:
def content_relevance(centroid,hypothesis_embeddings):
  d = {}
  for sentence,embed in hypothesis_embeddings.items():
    d[sentence] = cosine(centroid,embed)
  return d

In [ ]:
def sentence_novelty(content_relevance_score,hypothesis_embeddings):
  novel_sentences = {}
  TAU = 0.95
  for sent1,embed1 in hypothesis_embeddings.items():
    max_similarity = 0
    for sent2,embed2 in hypothesis_embeddings.items():
      if sent1!=sent2 and cosine(embed1,embed2)>max_similarity:
          max_similarity = cosine(embed1,embed2)

    if max_similarity<TAU:
      novel_sentences[sent1] = 1

    if max_similarity>TAU:
      if content_relevance_score[sent1]>content_relevance_score[sent2]:
        novel_sentences[sent1] = 1
      else:
        novel_sentences[sent2] = 1

    else:
      novel_sentences[sent1] = 1-max_similarity
  return novel_sentences

In [ ]:
def sentence_position(hypothesis):
  score_sent = {}
  for i,sent in enumerate(hypothesis):
    score_sent[sent] = max(0.5,math.exp(-(i+1)/(len(hypothesis)**(1/3))))
  return score_sent

In [ ]:
def total_score(content_relevance_score,sentence_novelty_score,sentence_position_score,text):
  ALPHA = 0.6
  BETA = 0.2
  GAMMA = 0.2
  final_score = {}
  for sent in content_relevance_score:
    final_score[sent] = ALPHA*content_relevance_score[sent]+BETA*sentence_novelty_score[sent]+GAMMA*sentence_position_score[sent]
  final_score = {k: v for k, v in sorted(final_score.items(), key=lambda item: item[1],reverse=True)}
  summary = ""
  for sent in final_score:
    if len(summary)<len(text)//2: #making summary of approx half length.
      summary+=sent
  return summary

In [ ]:
def evaluate(predicted_summary,actual_summary):
  rouge = Rouge()
  score = rouge.get_scores(predicted_summary,actual_summary)
  print(score[0])

In [ ]:
text = train_source[3]
actual_summary = train_target[3]
predicted_summary = convert_to_embeddings(text)
print("Length of text ",len(text))
print("length of summary",len(predicted_summary))
print(predicted_summary)
evaluate(predicted_summary,actual_summary)

Length of text  2681
length of summary 1430
unfortunately for him , many people had already seen it and responded .he deleted the original ( which is why i had to link to a retweet ) obviously aware that what he had posted was wrong .unfortunately for him , many people had already seen ittucker carlson exposes his own sexism on twitter ( updated ) tucker carlson has done some good work in the past … his site , the daily caller , is a frequent stop of mine and many other conservatives .if you haven ’ t heard by now , monday evening , tucker carlson posted a disturbing tweet about governor palin which said : palin ’ s popularity falling in iowa , but maintains lead to become supreme commander of milfistan aside from tucker ’ s sheep-like response to warped poll numbers , he also failed to take ownership of his sexist comment .this is also why i am so angered by tucker ’ s recent actions .” but that wasn ’ t enough to spare him the ire of conservative women on the blogosphere and twitter 

In [ ]:
train_source[1]

'los angeles ( ap ) — in her first interview since the nba banned her estranged husband , shelly sterling says she will fight to keep her share of the los angeles clippers and plans one day to divorce donald sterling . ( click prev or next to continue viewing images. ) advertisement ( click prev or next to continue viewing images. ) los angeles clippers co-owner shelly sterling , below , watches the clippers play the oklahoma city thunder along with her attorney , pierce o \' donnell , in the first half of game 3 of the western conference ... ( associated press ) shelly sterling spoke to barbara walters , and abc news posted a short story with excerpts from the conversation sunday . nba commissioner adam silver has banned donald sterling for making racist comments and urged owners to force sterling to sell the team . silver added that no decisions had been made about the rest of sterling \' s family . according to abc \' s story , shelly sterling told walters : " i will fight that deci